In [1]:
import random
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import shapely.geometry as shp
import math
from matplotlib.mlab import griddata
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor
from matplotlib.colors import LinearSegmentedColormap
from shapely.geometry import Polygon, MultiPolygon

from geopandas import GeoDataFrame
import geopandas as gpd

%matplotlib inline  

In [7]:
airports_df = pd.read_csv("./processed/claimData.csv")
airports_df.dropna()
airports_df = airports_df.loc[airports_df['count']>250 ,:]
airports_df.to_csv('./viz/data/airports.csv', columns=['Airport.Code','Name','Lat','Lon'], index_label='id')
airports_df.head()

,Unnamed: 0,Airport.Code,Name,Lat,Lon,count,meanEnplaned,totalEnplaned,other,baggage/cases/purses,...,medical/science,food & drink,automobile parts & accessories,tools & home improvement supplies,sporting equipment & supplies,"books, magazines & other",home decor,outdoor items,travel accessories,crafting & hobby
2,3,ABQ,Albuquerque International Sunport Airport,35.040199,-106.609001,917,2.857646e+06,40007037.0,0,107,...,10,1,1,1,3,1,6,1,5,0
12,13,ALB,Albany International Airport,42.748299,-73.801697,924,1.345188e+06,18832632.0,0,142,...,15,3,0,2,4,1,2,3,5,1
16,17,ANC,Ted Stevens Anchorage International Airport,61.174400,-149.996002,970,2.396599e+06,33552392.0,0,94,...,9,5,0,5,8,2,3,3,4,2
18,19,ATL,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,5509,4.324062e+07,605368633.0,0,538,...,71,54,1,22,16,20,31,1,35,9
20,21,AUS,Austin Bergstrom International Airport,30.194500,-97.669899,931,4.217282e+06,59041954.0,0,74,...,15,1,0,1,2,4,7,1,5,1


In [8]:
airports_df.columns

Index(['Unnamed: 0', 'Airport.Code', 'Name', 'Lat', 'Lon', 'count',
       'meanEnplaned', 'totalEnplaned', 'other', 'baggage/cases/purses',
       'personal electronics', 'toys & games', 'office equipment & supplies',
       'clothing', 'computer & accessories', 'jewelry & watches',
       'personal accessories', 'cameras', 'audio/video',
       'cosmetics & grooming', 'household items',
       'musical instruments & accessories', 'hunting & fishing items',
       'medical/science', 'food & drink', 'automobile parts & accessories',
       'tools & home improvement supplies', 'sporting equipment & supplies',
       'books, magazines & other', 'home decor', 'outdoor items',
       'travel accessories', 'crafting & hobby'],
      dtype='object')

In [9]:
totalClaims_all = airports_df['count'].sum()
totalEnplaned_all = airports_df['totalEnplaned'].sum()
claimsPerEnplaned_all = totalClaims_all / (totalEnplaned_all)

print(claimsPerEnplaned_all)

1.7869757711902917e-05


In [43]:
winnerStats = []
categoryOfInterest = 'musical instruments & accessories'
airports_df['target'] = airports_df[categoryOfInterest]/(airports_df['totalEnplaned'])

In [44]:
#find max in this category
maxRow = airports_df['target'].idxmax()
thisWinnerData = airports_df.ix[maxRow]

thisWinner = {'idx':  0,
              'Catg': categoryOfInterest,
              'Code': thisWinnerData['Airport.Code'],
              'Name': thisWinnerData['Name'],
              'Lat':  thisWinnerData['Lat'],
              'Lon':  thisWinnerData['Lon']}
winnerStats.append(thisWinner)

## Build the even grid

In [11]:
# Create grid values first.
xi = np.linspace(-180, -65, 115*5+1)
yi = np.linspace(10, 75, 65*5+1)
Xi, Yi = np.meshgrid(xi, yi)

In [12]:
global myTrainingWeights
myTrainingWeights = np.array(airports_df['meanEnplaned'])

trainingX = np.column_stack((airports_df['Lon'], airports_df['Lat']))
trainingY = np.array(airports_df['target'])

#### Just do it manually with forloops 

In [13]:
N_neighbors_min = 40
R_max = 250

def greatCircleDistance(lon1,lat1,lon2,lat2):
    deg2rad = math.pi/180.0
    lon1r = lon1*deg2rad
    lat1r = lat1*deg2rad
    lon2r = lon2*deg2rad
    lat2r = lat2*deg2rad
    if (abs(lon1-lon2)+abs(lat1-lat2)>0.0000001):
        centralAngle = math.acos( math.sin(lat1r)*math.sin(lat2r) + math.cos(lat1r)*math.cos(lat2r)*math.cos(lon2r-lon1r)  )
    else:
        centralAngle = 0
    return centralAngle*6378

predCoords = np.column_stack((Xi.reshape(-1,1),Yi.reshape(-1,1)))
N_points = predCoords.shape[0]
N_training = trainingX.shape[0]
predVals = np.zeros(N_points)

for ii in range(N_points):
    thisPoint = predCoords[ii]
    
    distToTrainings = np.zeros(N_training)
    for jj in range(N_training):
        thisTraining = trainingX[jj]
        distToTrainings[jj] = greatCircleDistance(thisPoint[0],thisPoint[1],thisTraining[0],thisTraining[1])
        
    # weighted avg of all within R_max (unless < N_neighbors_min... in which case jsut weight nearest N )
    weightQuotients = myTrainingWeights / (distToTrainings**2)
    # which ones to use?
    # weighted avg of all within R_max (unless < N_neighbors_min... in which case jsut weight nearest N )
    withinRad = distToTrainings<R_max
    NumWithinRad = sum(withinRad)
    if (NumWithinRad >= N_neighbors_min):
        # weighted avg of all within R_max
        isUsed = withinRad
    else:
        # use the nearest N
        sortedDists = np.sort(distToTrainings)
        nthDist = sortedDists[N_neighbors_min]
        isUsed = 1.00*(distToTrainings<(0.999999*nthDist))
        
    myWeights = weightQuotients*isUsed
    #if (sum(myWeights)<1):
        #print(sortedDists)10
    predVals[ii] = np.average(trainingY, weights=myWeights)
    
    if (ii<10):
        print(weightQuotients.shape)
        print(NumWithinRad)
        print("--")

predVals.reshape(Xi.shape)
zi = predVals.reshape(Xi.shape)
print(zi.shape)

(95,)
0
--
(95,)
0
--
(95,)
0
--
(95,)
0
--
(95,)
0
--
(95,)
0
--
(95,)
0
--
(95,)
0
--
(95,)
0
--
(95,)
0
--
(326, 576)


### plot things

In [19]:
#prepare zi for plotting
gobalAvgValue = sum(airports_df[categoryOfInterest])/sum(airports_df['totalEnplaned'])
print(gobalAvgValue)
zi_diff = ((zi-gobalAvgValue)/gobalAvgValue)

4.88685945227e-08


In [ ]:
#fig, (ax1, ax2) = plt.subplots(nrows=2)
plt.figure(figsize=(20,10))
#ax1.contour(xi, yi, zi, 14, linewidths=0.5, colors='k')

maxDiff = 100.0
zi_diff_clipped = np.minimum(np.maximum(zi_diff,-maxDiff/100.0),maxDiff/100.0)
#cntr1 = ax1.contourf(xi, yi, zi_diff_clipped, np.linspace(-maxDiff/100.0, maxDiff/100.0, 51), cmap="RdBu_r")
mycolorlist = ['#2d4059','#4d5b71','#6c788a','#8d96a4','#afb5bf','#d2d5da','#f6f6f6','#ffddb5','#ffc585','#feab63','#f99151','#f2744e','#ea5455']
mycolorlist = ['#283c63','#4e5575','#717087','#938c99','#b5a9ac','#d8c9bf','#fbe8d3','#fed2c2','#ffbdb2','#ffa8a2','#fe9191','#fb7982','#f85f73']
mycolorlist = ['#537780','#6f8c90','#8ca2a1','#a7b8b1','#c4d0c2','#e1e7d3','#ffffe4','#defada','#bbf5d1','#96edc9','#70e6c3','#4bddbe','#11d3bc']
myLevels = np.linspace(-maxDiff/100.0, maxDiff/100.0, 101)
myplot = plt.contourf(xi, yi, zi_diff_clipped, myLevels, cmap=LinearSegmentedColormap.from_list("hi", mycolorlist, N=51))

#plt.set_size_inches(18.5, 20.5)

plt.colorbar(cntr1, ax=ax1)
plt.plot(airports_df['Lon'], airports_df['Lat'], 'ko', ms=3)
plt.axis((-135, -65, 25, 50))
#ax1.axis((-90, -80, 30, 40))
plt.title('grid and contour')

#plt.subplots_adjust(hspace=0.5)
figure.show()

In [ ]:
figure = plt.figure()
ax = figure.add_subplot(111)
mycontour = plt.contourf(xi, yi, zi_diff_clipped, myLevels, cmap=plt.cm.jet)

def collec_to_gdf(collec_poly):
    """Transform a `matplotlib.contour.QuadContourSet` to a GeoDataFrame"""
    polygons, colors, levels = [], [], []
    for i, polygon in enumerate(collec_poly.collections):
        mpoly = []
        for path in polygon.get_paths():
            try:
                path.should_simplify = False
                poly = path.to_polygons()
                # Each polygon should contain an exterior ring + maybe hole(s):
                exterior, holes = [], []
                if len(poly) > 0 and len(poly[0]) > 3:
                    # The first of the list is the exterior ring :
                    exterior = poly[0]
                    # Other(s) are hole(s):
                    if len(poly) > 1:
                        holes = [h for h in poly[1:] if len(h) > 3]
                mpoly.append(Polygon(exterior, holes))
            except:
                print('Warning: Geometry error when making polygon #{}'
                      .format(i))
        if len(mpoly) > 1:
            mpoly = MultiPolygon(mpoly)
            polygons.append(mpoly)
            colors.append(polygon.get_facecolor().tolist()[0])
            levels.append(myLevels[i]*100.0)
        elif len(mpoly) == 1:
            polygons.append(mpoly[0])
            colors.append(polygon.get_facecolor().tolist()[0])
            levels.append(myLevels[i]*100.0)
    return GeoDataFrame(
        geometry=polygons,
        data={'RGBA': colors, 'levels':levels},
        crs={'init': 'epsg:4326'})

contour_df = collec_to_gdf(mycontour)
#print(contour_df.geometry)

In [ ]:
us_df = gpd.read_file('us-states.json')
usaPoly = us_df.buffer(0).unary_union

In [ ]:
#res_intersection = gpd.overlay(contour_df, us_df, how='intersection')


N_shapes = len(contour_df.geometry)
for ii in range(N_shapes):
    oldGeom = contour_df.geometry[ii]
    newGeom = oldGeom.intersection(usaPoly)
    contour_df.geometry[ii] = newGeom
    


f = open("./demo.geojson", "w")
f.write(contour_df.to_json())


In [45]:

print(winnerStats)

[{'idx': 0, 'Catg': 'musical instruments & accessories', 'Code': 'ALB', 'Name': 'Albany International Airport', 'Lat': 42.748298645019496, 'Lon': -73.801696777343793}]


In [38]:
for idx in range(8,33):
    print('--')
    print(idx)
    categoryOfInterest = airports_df.columns[idx]
    print(categoryOfInterest)


--
8
other
--
9
baggage/cases/purses
--
10
personal electronics
--
11
toys & games
--
12
office equipment & supplies
--
13
clothing
--
14
computer & accessories
--
15
jewelry & watches
--
16
personal accessories
--
17
cameras
--
18
audio/video
--
19
cosmetics & grooming
--
20
household items
--
21
musical instruments & accessories
--
22
hunting & fishing items
--
23
medical/science
--
24
food & drink
--
25
automobile parts & accessories
--
26
tools & home improvement supplies
--
27
sporting equipment & supplies
--
28
books, magazines & other
--
29
home decor
--
30
outdoor items
--
31
travel accessories
--
32
crafting & hobby


In [50]:
with open("./viz/data/winners.json", 'w') as outfile:
        json.dump(winnerStats, outfile)